## Download the dataset

In [1]:
!pip install datasets
!pip install ipywidgets
!pip install --upgrade huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from huggingface_hub import notebook_login
# Notebook login via HF's token
notebook_login()

In [3]:
from datasets import *
import pandas as pd
test = pd.DataFrame(load_dataset("huhu2023/test-gold-huhu2023", split='test'))

In [4]:
test.head()

,index,tweet,humor,prejudice_woman,prejudice_lgbtiq,prejudice_inmigrant_race,gordofobia,mean_prejudice
0,52830,-Mamá en la escuela me dicen gorda -Pobresilla...,1,1,0,0,1,1.6
1,78883,"No te sientas diferente, da igual si eres negr...",1,0,0,1,1,1.4
2,78926,Si esta asi.. SUPER SI.. y que se pongan celos...,1,1,0,0,1,1.3
3,61844,—Bebé ¿Me veo gorda con este vestido?\n—¡No mi...,1,1,0,0,1,2.3
4,78830,Las mujeres solo desean 2 cosas en la vida: co...,1,1,0,0,1,2.4


In [5]:
from google.colab import drive
# mount your google drive
drive.mount('/content/drive')
PATH = "/content/drive/My Drive/Colab Notebooks/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
sub = pd.read_csv(PATH+"submission_bin.csv",  sep=',', on_bad_lines='skip') 

In [8]:
df = test[["index", "tweet", "humor"]]

In [9]:
sub.head()

,tweet_id,humour
0,52830,1
1,78883,1
2,78926,0
3,61844,1
4,78830,1


In [10]:
df2 = sub[["tweet_id", "humour"]]
df2 = df2.rename(columns={"tweet_id": "index"})

In [11]:
df2.head()

,index,humour
0,52830,1
1,78883,1
2,78926,0
3,61844,1
4,78830,1


In [65]:
result = pd.concat([df, df2], axis=1)
# result = result[result["humor"] != result["humour"]]

# IMPORTANT

As the results obtained with the above method get some weird results, I try a more manual approach to check if the library heviour is the same as I expected. 

You can skip the code until the next mark

In [61]:
results = []
for index, entry in df.iterrows():
  results.append({})
  # entry.concat(df2[df2["index"] == entry["index"]]["humour"])
  results[-1] = entry.to_dict()
  results[-1]["humour"] = df2[df2["index"] == entry["index"]].to_dict("records")[0]["humour"]  

In [64]:
total = []
for result in results:
  if result["humor"] != result["humour"]:
    total.append(result)


119

# End of the alternative branch

In [67]:
result = result[result["humor"] != result["humour"]]
result


119

In [ ]:
result.to_csv(f"{PATH}/bin_join.csv", encoding="utf-8")

# Check multilabel

In [ ]:
sub_multi = pd.read_csv(PATH+"submission-multi.csv",  sep=',', on_bad_lines='skip')
sub_multi = sub_multi.rename(columns={"tweet_id": "index"})

In [ ]:
labels = ["index", "tweet", "prejudice_woman", "prejudice_lgbtiq", "prejudice_inmigrant_race", "gordofobia"]
df_multi = test[labels]

In [ ]:
sub_multi = sub_multi[["index", "prejudice_group1", "prejudice_group2", "prejudice_group3", "prejudice_group4"]]

In [ ]:
result_multi = pd.concat([df_multi, sub_multi], axis=1)
result_multi

,index,tweet,prejudice_woman,prejudice_lgbtiq,prejudice_inmigrant_race,gordofobia,index,prejudice_group1,prejudice_group2,prejudice_group3,prejudice_group4
0,52830,-Mamá en la escuela me dicen gorda -Pobresilla...,1,0,0,1,52830,0,0,0,1
1,78883,"No te sientas diferente, da igual si eres negr...",0,0,1,1,78883,0,0,1,0
2,78926,Si esta asi.. SUPER SI.. y que se pongan celos...,1,0,0,1,78926,1,0,0,1
3,61844,—Bebé ¿Me veo gorda con este vestido?\n—¡No mi...,1,0,0,1,61844,0,0,0,1
4,78830,Las mujeres solo desean 2 cosas en la vida: co...,1,0,0,1,78830,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
773,9496,Decir que una mujer está soltera es de machist...,1,0,0,0,9496,1,0,0,0
774,14026,¿cómo un aliado se atreve a chamuyar a una ant...,1,0,0,0,14026,1,0,0,0
775,12393,"MENTION No hicieron nada por las mujeres, son ...",1,0,0,0,12393,1,0,0,0
776,18723,Cuando llegará ese día en que las chicas organ...,1,0,0,0,18723,1,0,0,0


In [ ]:
results_multi = result_multi[
    (result_multi["prejudice_woman"] != result_multi["prejudice_group1"]) |
    (result_multi["prejudice_lgbtiq"] != result_multi["prejudice_group2"]) |
    (result_multi["prejudice_inmigrant_race"] != result_multi["prejudice_group3"]) |
    (result_multi["gordofobia"] != result_multi["prejudice_group4"])
    ]

In [ ]:
results_multi.to_csv(f"{PATH}/multi_join.csv", encoding="utf-8")